# Data Cleaning & Processing

This notebook contains preprocessing steps for the data.  These include changing datatypes and standardizing missing values.

The steps taken here include:
* Import original data
* Unpack the nested structure of certain columns
* Group unique categorical values
* Deal with missing values

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas import json_normalize
import datetime as dt
import math

In [ ]:
df = pd.read_csv('../Data/train_v2.csv', dtype={'fullVisitorId': 'str'})

This project only uses data on visits from customers located in the US, so we filter our data here.  Because of memory limitations, we will save our filtered data as a csv, then load that csv again in the next step.

In [ ]:
df_us = df.loc[df['geoNetwork'].astype(str).str.contains('United States')]
df_us.to_csv('../Data/data1.csv')

The original dataset has some columns with nested data structures.  Here we define a function to unpack those nested structures into separate columns.

In [2]:
def unpack_df(csv_path='../Data/data1.csv', nrows=None):
    """
    Unpack the nested structure of JSON columns
    
    *Source*
    Much of the code here was created by Kaggler Julian Peller
    https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields
    ***
    
    Input:
    csv_path -- Path to csv file to unpack
    nrows -- Number of rows to unpack
    
    Output:
    Returns a copy of the csv as a dataframe with each nested element as a new column.
    """
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
df = unpack_df()
df.drop('Unnamed: 0', axis=1, inplace=True)

Loaded data1.csv. Shape: (717217, 60)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717217 entries, 0 to 717216
Data columns (total 59 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    717217 non-null  object
 1   customDimensions                                   717217 non-null  object
 2   date                                               717217 non-null  int64 
 3   fullVisitorId                                      717217 non-null  object
 4   hits                                               717217 non-null  object
 5   socialEngagementType                               717217 non-null  object
 6   visitId                                            717217 non-null  int64 
 7   visitNumber                                        717217 non-null  int64 
 8   visitStartTime                                     717217 non-null  int64 
 9   devi

Some columns are filled with 'data' that is not actually available.  All values in these columns are simply 'not available in demo dataset'.  Here we remove these columns.

In [5]:
df.drop(['customDimensions', 'device.browserSize', 'device.browserVersion', 'device.flashVersion'], axis=1, inplace=True)

In [6]:
for column in df.columns:
    if df[column].value_counts().index[0] == 'not available in demo dataset':
        df.drop(column, axis=1, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717217 entries, 0 to 717216
Data columns (total 38 columns):
 #   Column                                        Non-Null Count   Dtype 
---  ------                                        --------------   ----- 
 0   channelGrouping                               717217 non-null  object
 1   date                                          717217 non-null  int64 
 2   fullVisitorId                                 717217 non-null  object
 3   hits                                          717217 non-null  object
 4   socialEngagementType                          717217 non-null  object
 5   visitId                                       717217 non-null  int64 
 6   visitNumber                                   717217 non-null  int64 
 7   visitStartTime                                717217 non-null  int64 
 8   device.browser                                717217 non-null  object
 9   device.operatingSystem                        717217 non-nu

In [8]:
df.head()

,channelGrouping,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,...,trafficSource.source,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd
0,Referral,20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Chrome,Chrome OS,...,sites.google.com,referral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Direct,20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Chrome,Android,...,(direct),(none),True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Referral,20171016,0166277907528479249,"[{'hitNumber': '1', 'time': '0', 'hour': '16',...",Not Socially Engaged,1508196701,1,1508196701,Chrome,Macintosh,...,(direct),(none),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20171016,5983987899933814948,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,1508177658,1,1508177658,Safari,iOS,...,google,organic,NaN,(not provided),NaN,NaN,NaN,NaN,NaN,NaN
4,Paid Search,20171016,6135613929977117121,"[{'hitNumber': '1', 'time': '0', 'hour': '8', ...",Not Socially Engaged,1508166431,10,1508166431,Safari,iOS,...,google,cpc,NaN,(Remarketing/Content targeting),Placement Accessories 300 x 250,NaN,NaN,NaN,NaN,NaN


Only a few of the columns are registering as numerical values.  We update the data types below.

In [9]:
df.describe()

,date,visitId,visitNumber,visitStartTime
count,7.172170e+05,7.172170e+05,717217.000000,7.172170e+05
mean,2.017048e+07,1.499216e+09,3.089140,1.499216e+09
std,6.235211e+03,1.582198e+07,11.948716,1.582198e+07
min,2.016080e+07,1.470035e+09,1.000000,1.470035e+09
25%,2.017012e+07,1.485273e+09,1.000000,1.485273e+09
50%,2.017073e+07,1.501174e+09,1.000000,1.501174e+09
75%,2.017121e+07,1.512548e+09,2.000000,1.512548e+09
max,2.018043e+07,1.525158e+09,457.000000,1.525158e+09


## Change Data Types

### 'date' to DateTime

In [10]:
df['date'] = df['date'].astype(str)
df['date'].value_counts()


20171212    4690
20171213    2777
20171004    2569
20170920    2444
20180327    2421
            ... 
20170506     495
20170513     491
20170415     489
20160904     487
20170416     472
Name: date, Length: 638, dtype: int64

In [11]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [12]:
df['visitStartTime']

0         1508176307
1         1508201613
2         1508196701
3         1508177658
4         1508166431
             ...    
717212    1483561580
717213    1483533179
717214    1483600246
717215    1483556333
717216    1483570454
Name: visitStartTime, Length: 717217, dtype: int64

### 'visitStartTime' to datetime

In [13]:
df['time'] = df['visitStartTime'].apply(lambda x: dt.datetime.utcfromtimestamp(float(x)))

### 'visitStartTime' into bins

Knowing what 'chunk' of a day a visit was made in could be more helpful than knowing the exact time.  Here we bin the time of the visit into 8 sections corresponding to the time of day.

In [14]:
bin_values = [0,4,8,12,16,20,24]
bin_labels = ['Late Night', 'Early Morning','Morning','Afternoon','Evening','Night']
df['hour'] = df['time'].dt.hour
df['time_of_day'] = pd.cut(df['hour'], bins=bin_values, labels=bin_labels, include_lowest=True)
df.drop(['hour', 'visitStartTime'], axis=1, inplace=True)

## Standardize Categorical Variable Values

In this section we standardize missing values and group together categorical variables that don't occur often.  This is because our One Hot Encoded dataframe we will use for modeling would become to large and sparse if we kept each unique value as its own column.

Examine value counts for all columns.  Group together unique values (<500 count) into 'Other' for ease of modeling.  Also group missing values into 'None'

In [16]:
value_counts_list = []
for column in df.columns:
    value_counts_list.append(df[column].value_counts())

### Group device_browser values

Keeping top 10 browsers.  Convert the rest to 'Other':

In [17]:
top_browsers = list(value_counts_list[7].index[0:10])
df['device.browser'] = df['device.browser'].apply(lambda x: x if x in top_browsers else 'Other')
df['device.browser'].value_counts()

Chrome               529385
Safari               127216
Firefox               15247
Internet Explorer     11974
Edge                   8420
Safari (in-app)        7369
Samsung Internet       7169
Android Webview        6242
Opera                  2165
Amazon Silk            1070
Other                   960
Name: device.browser, dtype: int64

### Group device_operatingSystem values

In [18]:
top_os = list(value_counts_list[8].index[0:6])
df['device.operatingSystem'] = df['device.operatingSystem'].apply(lambda x: x if x in top_os else 'Other')
df['device.operatingSystem'].value_counts()

Macintosh    230058
Windows      160625
iOS          123967
Android      116485
Linux         46001
Chrome OS     39310
Other           771
Name: device.operatingSystem, dtype: int64

### Group ad content values

In [19]:
top_ad_content = list(value_counts_list[31].index[0:9])
df['trafficSource.adContent'] = df['trafficSource.adContent'].apply(lambda x: x if x in top_ad_content else 'Other')
df['trafficSource.adContent'].value_counts()

Other                             673848
Google Merchandise Store           27999
Google Merchandise Collection       6509
Placement Accessores 300 x 250      2680
Smart display ad - 8/17/2017        2135
Google Online Store                 1534
Official Google Merchandise          784
Full auto ad IMAGE ONLY              675
Placement Drinkware 300x250          544
Display Ad created 3/11/14           509
Name: trafficSource.adContent, dtype: int64

### Group ad campaign values

In [20]:
top_camp = list(value_counts_list[26].index[0:8])
df['trafficSource.campaign'] = df['trafficSource.campaign'].apply(lambda x: x if x in top_camp else 'Other')
df['trafficSource.campaign'].replace('(not set)', 'None', inplace=True)
df['trafficSource.campaign'].value_counts()

None                                                                    653237
1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS                   15396
AW - Dynamic Search Ads Whole Site                                       14679
1000557 | GA | US | en | Hybrid | GDN Remarketing                        12598
AW - Accessories                                                          7410
Data Share Promo                                                          6983
Other                                                                     3782
Smart Display Campaign                                                    2135
"google + redesign/Accessories March 17" All Users Similar Audiences       997
Name: trafficSource.campaign, dtype: int64

### Group keyword values

In [21]:
top_keyword = list(value_counts_list[30].index[0:11])
df['trafficSource.keyword'] = df['trafficSource.keyword'].apply(lambda x: x if x in top_keyword else 'Other')
df['trafficSource.keyword'].replace('(not provided)', 'None', inplace=True)
df['trafficSource.keyword'].value_counts()

Other                              433603
None                               228509
(User vertical targeting)           16069
(automatic matching)                15309
6qEhsCssdK0z36ri                    10525
(Remarketing/Content targeting)      3520
1hZbAqLCbjwfgOH7                     2889
google merchandise store             2229
Google Merchandise                   1810
1X4Me6ZKNV0zg-jV                     1256
google store                          797
DoubleClick Ad Exchange               701
Name: trafficSource.keyword, dtype: int64

### Consolidate 'None' medium values

In [22]:
df['trafficSource.medium'].replace('(none)', 'None', inplace=True)
df['trafficSource.medium'].replace('(not set)', 'None', inplace=True)

### Group referral path values

In [23]:
top_path = list(value_counts_list[25].index[0:22])
df['trafficSource.referralPath'] = df['trafficSource.referralPath'].apply(lambda x: x if x in top_path else 'Other')
df['trafficSource.referralPath'].replace('/', 'None', inplace=True)
df['trafficSource.referralPath'].value_counts()

Other                                                                                                            541749
None                                                                                                             117629
/yt/about/                                                                                                        17610
/offer/2145                                                                                                        8110
/analytics/web/                                                                                                    6235
/a/google.com/googletopia/discounts-deals-and-free-stuff/alphabet-google-discounts                                 5518
/permissions/using-the-logo.html                                                                                   2321
/yt/creators/                                                                                                      2019
/yt/about/copyright/                    

### Group source values

In [24]:
top_source = list(value_counts_list[27].index[0:22])
df['trafficSource.source'] = df['trafficSource.source'].apply(lambda x: x if x in top_source else 'Other')
df['trafficSource.source'].value_counts()

(direct)                       334176
google                         293586
youtube.com                     35234
sites.google.com                 8152
analytics.google.com             7130
Partners                         6988
dfa                              6114
Other                            5068
m.facebook.com                   2751
google.com                       2726
mail.google.com                  1910
groups.google.com                1822
reddit.com                       1726
bing                             1711
siliconvalley.about.com          1389
yahoo                            1214
googleads.g.doubleclick.net      1040
facebook.com                     1010
t.co                              886
baidu                             783
l.facebook.com                    655
blog.golang.org                   580
dealspotr.com                     566
Name: trafficSource.source, dtype: int64

### Remove social engagement

No socially engaged observations, so remove the column.

In [25]:
df.drop('socialEngagementType', axis=1, inplace=True)

### Remove totals bounces

All observations one value, so remove the column.

In [26]:
df.drop('totals.bounces', axis=1, inplace=True)

### Remove total new visits

All observations one value, so remove the column.

In [27]:
df.drop('totals.newVisits', axis=1, inplace=True)

### Remove total visits

All observations one value, so remove the column.

In [28]:
df.drop('totals.visits', axis=1, inplace=True)

### Remove gcl ID

Unknown meaning, so remove the column.

In [29]:
df.drop('trafficSource.adwordsClickInfo.gclId', axis=1, inplace=True)

### Remove video ad indicator

No true values, so remove the column.

In [30]:
df.drop('trafficSource.adwordsClickInfo.isVideoAd', axis=1, inplace=True)

### Remove direct traffic source

No false observations, so remove the column.

In [31]:
df.drop('trafficSource.isTrueDirect', axis=1, inplace=True)

# Resolve NaN's

Some columns in the 'totals' and 'trafficSource' categories have null values that must be filled in or removed before modeling can be done.

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717217 entries, 0 to 717216
Data columns (total 32 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   channelGrouping                               717217 non-null  object        
 1   date                                          717217 non-null  datetime64[ns]
 2   fullVisitorId                                 717217 non-null  object        
 3   hits                                          717217 non-null  object        
 4   visitId                                       717217 non-null  int64         
 5   visitNumber                                   717217 non-null  int64         
 6   device.browser                                717217 non-null  object        
 7   device.operatingSystem                        717217 non-null  object        
 8   device.isMobile                               717217 n

Give session quality missing values their own value of -1

In [33]:
#Replace missing values with -1
df['totals.sessionQualityDim'].fillna(-1, inplace=True)
df['totals.sessionQualityDim'].value_counts()

-1     333031
1      263525
2       41341
3       12665
4        6638
        ...  
96         52
97         27
98          9
99          5
100         2
Name: totals.sessionQualityDim, Length: 101, dtype: int64

If revenue data is missing, assume that no purchase was made.

In [34]:
# Fill in 0s for transaction/revenue data
df['totals.totalTransactionRevenue'].fillna(0.0, inplace=True)
df['totals.transactionRevenue'].fillna(0.0, inplace=True)
df['totals.transactions'].fillna(0.0, inplace=True)

### Fill Missing values for timeOnSite with mean for purchaser group

Note that this will influence average time on site values in the EDA and modeling processes.

In [35]:
# Fill time on site NaN's using mean.  Different for purchasers vs not.
df['totals.timeOnSite'].value_counts()

4       5021
5       4697
6       4672
10      4419
9       4369
        ... 
4624       1
3484       1
6147       1
5515       1
2502       1
Name: totals.timeOnSite, Length: 4243, dtype: int64

In [36]:
no_purchase_tos = df.loc[df['totals.totalTransactionRevenue'] == 0.0]['totals.timeOnSite']
no_purchase_tos.dropna(inplace=True)
no_purchase_time = no_purchase_tos.astype(int).mean()

purchase_tos = df.loc[df['totals.totalTransactionRevenue'] != 0.0]['totals.timeOnSite']
purchase_tos.dropna(inplace=True)
purchase_time = purchase_tos.astype(int).mean()

<ipython-input-36-a71f0dff90ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_purchase_tos.dropna(inplace=True)
<ipython-input-36-a71f0dff90ac>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_tos.dropna(inplace=True)


In [37]:
no_purchase_time

239.50996917387408

In [38]:
purchase_time

997.2024018580411

In [39]:
# Create categorical var for purchase vs not
df.loc[df['totals.transactionRevenue'].astype(int) > 0, 'purchase'] = True
df.loc[df['totals.transactionRevenue'].astype(int) == 0, 'purchase'] = False
df['purchase'] = df['purchase'].astype(bool)

In [52]:
df['totals.timeOnSite'].fillna(df['purchase'].apply(lambda x: purchase_time if x==True else no_purchase_time), inplace=True)

In [53]:
df['totals.timeOnSite'].value_counts()

239.50996917387408    278812
4                       5021
5                       4697
6                       4672
10                      4419
                       ...  
3484                       1
6147                       1
5391                       1
5515                       1
2502                       1
Name: totals.timeOnSite, Length: 4245, dtype: int64

### Fill page views NaN's with 1

If a visit to the site was made, then at least one page must have been viewed, so we fill the missing values with 1.

In [42]:
df['totals.pageviews'].value_counts()

1      279242
2       94384
3       73996
4       49348
5       38224
        ...  
219         1
340         1
190         1
500         1
223         1
Name: totals.pageviews, Length: 198, dtype: int64

In [43]:
df['totals.pageviews'].fillna(1.0, inplace=True)

### Fill AdWords NaN's

Give missing values their own category called 'None'

In [44]:
df['trafficSource.adwordsClickInfo.adNetworkType'].value_counts()

Google Search      31608
Content            30129
Search partners        8
Name: trafficSource.adwordsClickInfo.adNetworkType, dtype: int64

In [45]:
df['trafficSource.adwordsClickInfo.adNetworkType'].fillna('None', inplace=True)

In [46]:
df['trafficSource.adwordsClickInfo.slot'].value_counts()

Top                       31020
RHS                       30671
Google Display Network       54
Name: trafficSource.adwordsClickInfo.slot, dtype: int64

In [47]:
df['trafficSource.adwordsClickInfo.slot'].fillna('None', inplace=True)

In [48]:
df['trafficSource.adwordsClickInfo.page'].value_counts()

1     60475
2       983
3       167
4        72
5        27
6         9
7         6
9         3
12        1
8         1
14        1
Name: trafficSource.adwordsClickInfo.page, dtype: int64

In [49]:
df['trafficSource.adwordsClickInfo.page'].fillna(0.0, inplace=True)

# Convert totals columns to integers

This is a necessary step for modeling.

In [55]:
for column in df.columns:
    if 'totals.' in str(column):
        df[column] = df[column].astype(int)

# Transform Revenue to Dollars

Original data has revenue data stored according to the following formula: Revenue = USD * 10^6.  Here we will convert revenue numbers back to USD.

In [56]:
df['revenue_usd'] = df['totals.transactionRevenue'] / (10**6)
df['total_revenue_usd'] = df['totals.totalTransactionRevenue'] / (10**6)

In [57]:
df.describe()

,visitId,visitNumber,totals.hits,totals.pageviews,totals.timeOnSite,totals.sessionQualityDim,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,revenue_usd,total_revenue_usd
count,7.172170e+05,717217.000000,717217.000000,717217.000000,717217.000000,717217.000000,717217.000000,7.172170e+05,7.172170e+05,717217.000000,717217.000000
mean,1.499216e+09,3.089140,6.117885,5.016974,257.966233,3.036248,0.025584,3.064463e+06,3.478864e+06,3.064463,3.478864
std,1.582198e+07,11.948716,11.130350,7.962957,383.534894,12.395530,0.169631,6.878500e+07,1.046080e+08,68.784999,104.607987
min,1.470035e+09,1.000000,1.000000,1.000000,1.000000,-1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.485273e+09,1.000000,1.000000,1.000000,61.000000,-1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
50%,1.501174e+09,1.000000,2.000000,2.000000,239.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
75%,1.512548e+09,2.000000,6.000000,6.000000,239.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
max,1.525158e+09,457.000000,500.000000,500.000000,19017.000000,100.000000,25.000000,2.312950e+10,4.708206e+10,23129.500000,47082.060000


In [58]:
df.loc[df['revenue_usd'] != 0.0].describe()

,visitId,visitNumber,totals.hits,totals.pageviews,totals.timeOnSite,totals.sessionQualityDim,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,revenue_usd,total_revenue_usd
count,1.765800e+04,17658.000000,17658.000000,17658.000000,17658.000000,17658.000000,17658.000000,1.765800e+04,1.765800e+04,17658.000000,17658.000000
mean,1.496104e+09,4.055895,34.498584,26.414543,997.202345,27.389115,1.036471,1.244697e+08,1.413014e+08,124.469656,141.301404
std,1.556898e+07,13.195237,26.241472,17.899671,859.952854,35.386088,0.342767,4.208008e+08,6.519319e+08,420.800829,651.931857
min,1.470051e+09,1.000000,2.000000,2.000000,9.000000,-1.000000,1.000000,1.000000e+04,1.200000e+06,0.010000,1.200000
25%,1.481919e+09,1.000000,18.000000,15.000000,430.000000,-1.000000,1.000000,2.298000e+07,2.888250e+07,22.980000,28.882500
50%,1.495665e+09,2.000000,27.000000,22.000000,735.000000,-1.000000,1.000000,4.580000e+07,5.270000e+07,45.800000,52.700000
75%,1.508048e+09,4.000000,43.000000,32.000000,1285.750000,66.000000,1.000000,9.999000e+07,1.079900e+08,99.990000,107.990000
max,1.525148e+09,430.000000,500.000000,466.000000,15047.000000,100.000000,25.000000,2.312950e+10,4.708206e+10,23129.500000,47082.060000


In [60]:
df.to_csv('../Data/data2.csv')

# Unpack 'hits' Column

The 'hits' column is stored in a more deeply nested data structure than the other columns, so it requires more detailed functions to unpack it.

All code in the following cell was provided by Kaggler Changhao Lee:
https://www.kaggle.com/leechh/a-way-to-input-all-columns-include-hits

In [2]:
def todict(dic, key, value):
    if key in dic:
        dic[key].append(value)
    else:
        dic[key] = [value]
    return dic


def resolve_json(hitsdic, hits_json, key='NoneName'):
    if type(hits_json) == list:
        if len(hits_json) == 0:
            pass
        else:
            for subjson in hits_json:
                hitsdic = resolve_json(hitsdic, subjson)
    elif type(hits_json) == dict:
        for i in hits_json.keys():
            hitsdic = resolve_json(hitsdic, hits_json[i],i)
    else:
        hitsdic = todict(hitsdic, key, hits_json)
    return hitsdic


def complex_replace(x):
    dic = {}
    return resolve_json(dic, json.loads(x.replace('\'','\"'). \
                                        replace('TRUE','true'). \
                                        replace('True','true'). \
                                        replace('FALSE','false'). \
                                        replace('False','false'). \
                                        replace(', \"',', !&~'). \
                                        replace('\", ','!&~, '). \
                                        replace('\": ','!&~: '). \
                                        replace(': \"',': !&~'). \
                                        replace(' {\"',' {!&~'). \
                                        replace('\"}, ','!&~}, '). \
                                        replace('[{\"','[{!&~'). \
                                        replace('\"}]','!&~}]'). \
                                        replace('\"','_'). \
                                        replace('!&~','\"'). \
                                        encode('gbk','ignore'). \
                                        decode('utf-8','ignore'). \
                                        replace('\\','')))


def replace(x):
    return  json.loads(x)


def load_hits_df(csv_path, nrows=None, chunksize=10_000, percent=100):
    n=1
    df_list = []
    feature = ['hits']
    chunk = pd.read_csv(csv_path,
                        nrows=nrows, 
                        chunksize=chunksize, 
                        dtype={'fullVisitorId': 'str'}) # Important!!
    for subchunk in chunk:
        for column in feature:
            if column in ['hits']:
                column_as_df = json_normalize(subchunk[column].apply(complex_replace))
            else:
                column_as_df = json_normalize(subchunk[column].apply(replace))
            column_as_df.columns = [f'{column}_{subcolumn}' for subcolumn in column_as_df.columns]
            subchunk.drop(column, axis=1, inplace=True)
            subchunk = subchunk.reset_index(drop=True).merge(column_as_df,
                                           right_index=True,
                                           left_index=True)
        n = n+1
        df_list.append(subchunk.astype('str'))
        del column_as_df, subchunk
    return pd.concat(df_list, ignore_index=True, sort=True)

In [3]:
hits = load_hits_df('../Data/data2.csv',nrows=None)


Save the unpacked and processed data file.

In [4]:
hits.to_csv('../Data/data3.csv')